In [3]:
import json
import requests
import env

class Data:
    METABASE_SESSION = ""
def getMetabaseSession(username, password):
    payload = {
        "username": username,
        "password": password
    }
    headers = {
        'content-type': "application/json",
    }
    response = requests.request("POST", env.METABASE_API_GET_SESSION, data=json.dumps(payload, separators=(',', ':')), headers=headers)
    Data.METABASE_SESSION = json.loads(response.text)['id']

def getMetaCards(card_ids):
    responses = []
    for card_id in card_ids:
        url = "http://metabase.jabama.com/api/card/" + str(card_id) + "/query"

        headers = {
            'Content-Type': "application/json",
            'X-Metabase-Session': Data.METABASE_SESSION,
            }

        responses.append([card_id, requests.request("POST", url, headers=headers)])
    return responses

def getDataFromTableResponses(responses):
    responses_data = []
    print(responses)
    for response in responses:
        rows = json.loads(response[1].text)['data']['rows']
        card_id = response[0]
        responses_data.append({
            "card_id": card_id,
            "rows": rows
        })
        
    return responses_data

def getDataFromPivotResponses(responses):
    responses_data = []
    
    for response in responses:
        cols = json.loads(response[1].text)['data']['cols']
        rows = json.loads(response[1].text)['data']['rows']
        card_id = response[0]
        col_titles = []
        for col in cols:
            col_titles.append(col['name'])

        responses_data.append({
            "card_id": card_id,
            "columns": col_titles,
            "rows": rows
        })
    return responses_data

def metabot_job():
 
    #Get and store the raw data of union of questions from Metabase.
#     pivot_metacard_responses = getMetaCards([env.CARD_ID_JOBAUDIT])
#     pivot_responses_data = getDataFromPivotResponses(pivot_metacard_responses)
    
    table_metacard_responses = getMetaCards([env.CARD_ID_JOBAUDIT])
    table_responses_data = getDataFromTableResponses(table_metacard_responses)
    print(table_responses_data)
    #End of creating the message blocks
    
    
def metabase_get_session_job():
    getMetabaseSession(env.METABASE_USERNAME, env.METABASE_PASSWORD)

metabase_get_session_job()
metabot_job()

[['1596', <Response [403]>]]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)